In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/iamdipsan/100-Days-ML/refs/heads/main/day-28/covid_toy.csv")

In [3]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [5]:
X=df.drop(columns='has_covid')
y=df['has_covid']

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [7]:
### Column transformer

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
transformer= ColumnTransformer(
    transformers=[
        ('tnf1',SimpleImputer(),['fever']),
        ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
        ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city']),

    ],remainder='passthrough'
)

In [8]:
# Fit and transform the data
transformer.fit(X_train)
X_train_transformed = transformer.transform(X_train)
X_test_transformed = transformer.transform(X_test)

In [9]:
feature_names = transformer.get_feature_names_out()

In [10]:
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=feature_names)
X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=feature_names)

In [11]:
X_train_transformed_df

,tnf1__fever,tnf2__cough,tnf3__gender_Male,tnf3__city_Delhi,tnf3__city_Kolkata,tnf3__city_Mumbai,remainder__age
0,99.0,0.0,0.0,0.0,0.0,0.0,22.0
1,104.0,1.0,0.0,0.0,0.0,0.0,56.0
2,98.0,0.0,0.0,0.0,1.0,0.0,31.0
3,104.0,1.0,0.0,1.0,0.0,0.0,75.0
4,99.0,0.0,1.0,0.0,0.0,0.0,72.0
...,...,...,...,...,...,...,...
75,101.0,1.0,0.0,0.0,1.0,0.0,51.0
76,99.0,0.0,1.0,0.0,0.0,0.0,65.0
77,104.0,0.0,1.0,0.0,0.0,1.0,42.0
78,104.0,0.0,0.0,0.0,0.0,0.0,18.0


In [12]:
y_train = y_train.map({'No': 0, 'Yes': 1})
y_test = y_test.map({'No': 0, 'Yes': 1})


In [13]:
X_train_transformed_df

,tnf1__fever,tnf2__cough,tnf3__gender_Male,tnf3__city_Delhi,tnf3__city_Kolkata,tnf3__city_Mumbai,remainder__age
0,99.0,0.0,0.0,0.0,0.0,0.0,22.0
1,104.0,1.0,0.0,0.0,0.0,0.0,56.0
2,98.0,0.0,0.0,0.0,1.0,0.0,31.0
3,104.0,1.0,0.0,1.0,0.0,0.0,75.0
4,99.0,0.0,1.0,0.0,0.0,0.0,72.0
...,...,...,...,...,...,...,...
75,101.0,1.0,0.0,0.0,1.0,0.0,51.0
76,99.0,0.0,1.0,0.0,0.0,0.0,65.0
77,104.0,0.0,1.0,0.0,0.0,1.0,42.0
78,104.0,0.0,0.0,0.0,0.0,0.0,18.0


In [14]:
y_train

,has_covid
43,1
62,1
3,0
71,0
45,0
...,...
96,1
67,0
64,0
47,0


In [16]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(X_train_transformed,y_train)

RandomForestClassifier()

In [17]:
y_pred=rfc.predict(X_test_transformed)

In [18]:
from sklearn.metrics import accuracy_score
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))

Random Forest Accuracy: 0.35


In [19]:
from sklearn.linear_model import LogisticRegression

lr= LogisticRegression(random_state=42)

lr.fit(X_train_transformed,y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [20]:
y_pred_lr = lr.predict(X_test_transformed)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))

Logistic Regression Accuracy: 0.35


In [24]:
import numpy as np

# Feature order based on your transformed dataset
sample = [102,      # fever
          0,        # cough ('Mild')
          1,        # gender_Male
          1, 0, 0,  # city_Delhi=1, Kolkata=0, Mumbai=0
          35]       # age

sample_array = np.array(sample).reshape(1, -1)

# Predict using your trained model (e.g., rfc)
prediction = rfc.predict(sample_array)
print("Prediction:", prediction[0])


Prediction: 0


In [27]:
import numpy as np

# Feature order based on your transformed dataset
sample = [102,      # fever
          0,        # cough ('Mild')
          1,        # gender_Male
          1, 0, 0,  # city_Delhi=1, Kolkata=0, Mumbai=0
          35]       # age

sample_array = np.array(sample).reshape(1, -1)

# Predict using your trained model (e.g., rfc)
prediction = rfc.predict(sample_array)
print("Prediction:", prediction[0])


Prediction: 0
